<a href="https://colab.research.google.com/github/brayanricardo13/Analisis-Econometrico/blob/main/Analista_de_Renta_fija.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
from datetime import datetime

# Simulando tus tablas como DataFrames
data1 = {
    'Inversionista': ['Ana', 'Pedro', 'Jose', 'Juan', 'Felipe'],
    'Fecha de compra': ['12/5/2021', '1/17/2022', '2/18/2022', '3/1/2023', '4/5/2023'],
    'Valor Nominal Titulo': [500000000.0, 500000000.0, 1500000000.0, 500000000.0, 500000000.0],
    'Precio compra': [1.01, 1.0235, 1.0, 0.9, 0.924]
}

data2 = {
    'FechaRegistro': [20230814],
    'Especie': ['BBPO520SB084'],
    'Tasaref': [0.0629],
    'Spread': [None],
    'DiasVcto': [1269],
    'Tasa': [0.1303],
    'Precio': [0.8262],
    'PreLimpio': [0.8245],
    'Duracion': [3.1099],
    'Modalidad': ['V'],
    'FechaEmision': [20200204],
    'FechaVencimiento': [20270204]
}

data3 = {
    'NEMO NUEVO': ['BBPO520SB084'],
    'EMISOR': ['BCO. POPULAR'],
    'TITULO': ['BONOS ORDINARIOS'],
    'MONEDA': ['COP'],
    'CUPON': [0.0629],
    'TASA REFERENCIA': ['FS'],
    'TASA BASE': [365],
    'PERIODICIDAD': ['TV'],
    'FECHA EMISION': ['2/4/2020'],
    'FECHA VTO': ['2/4/2027']
}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
df3 = pd.DataFrame(data3)

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy.optimize import newton

def add_months(date, months):
    month = date.month - 1 + months
    year = date.year + month // 12
    month = month % 12 + 1
    day = min(date.day, [31, 29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31][month - 1])
    return date.replace(year=year, month=month, day=day)

def xirr(transactions):
    days = [(date - transactions[0][0]).days for date, _ in transactions]
    totals = [total for _, total in transactions]

    def present_value(rate):
        total_pv = sum([total / (1 + rate)**(day / 365.0) for day, total in zip(days, totals)])
        return total_pv

    irr = newton(lambda r: present_value(r), 0.1)
    return irr

def fechas_pago_cupon(fecha_compra_str, fecha_emision_str, fecha_registro_str, periodicidad, tasa_base, spread, precio, precio_compra, tasa):
    periodos_por_año = {
        'TV': 4
    }

    n = periodos_por_año.get(periodicidad, None)
    if n is None:
        return "Periodicidad no reconocida"

    fecha_compra = datetime.strptime(fecha_compra_str, '%m/%d/%Y').date()
    fecha_emision = datetime.strptime(fecha_emision_str, '%m/%d/%Y').date()
    fecha_registro = datetime.strptime(str(fecha_registro_str), '%Y%m%d').date()

    fechas_pago = [fecha_compra]

    while fecha_emision < fecha_registro:
        if fecha_emision > fecha_compra:
            fechas_pago.append(fecha_emision)
        fecha_emision = add_months(fecha_emision, 12 // n)

    fechas_pago.append(fecha_registro)

    dias = [0]
    for i in range(1, len(fechas_pago)):
        diferencia = (fechas_pago[i] - fechas_pago[i-1]).days
        dias.append(diferencia)

    TP = (1 + spread)**(1/n) - 1
    flujos = [precio_compra * -100]
    flujos.extend([TP * 100] * (len(fechas_pago) - 2))
    flujos.append(TP * 100 + precio * 100)

    TP_tasa = (1 + tasa)**(1/n) - 1

    df = pd.DataFrame({
        'Conteo de Flujos': range(len(fechas_pago)),
        'Fecha de Pago': fechas_pago,
        'Días': dias,
        'Días/Tasa Base': [dia / tasa_base for dia in dias],
        'Flujos': flujos
    })

    df['Valor Presente'] = df['Flujos'] / (1 + TP_tasa)**df['Días/Tasa Base']

    transactions = list(zip(df['Fecha de Pago'], df['Flujos']))
    tir_resultante = xirr(transactions)

    return df, tir_resultante

# Ejemplo de uso
ejemplo_df, tir_resultante = fechas_pago_cupon('2/18/2022', '2/4/2020', '20230814', 'TV', 365, 0.0629, 0.8262, 1, 0.1303)
print(ejemplo_df)
print(f"La Tasa Interna de Retorno (TIR) es: {tir_resultante * 100:.2f}%")


   Conteo de Flujos Fecha de Pago  Días  Días/Tasa Base      Flujos  \
0                 0    2022-02-18     0        0.000000 -100.000000   
1                 1    2022-05-04    75        0.205479    1.536713   
2                 2    2022-08-04    92        0.252055    1.536713   
3                 3    2022-11-04    92        0.252055    1.536713   
4                 4    2023-02-04    92        0.252055    1.536713   
5                 5    2023-05-04    89        0.243836    1.536713   
6                 6    2023-08-04    92        0.252055    1.536713   
7                 7    2023-08-14    10        0.027397   84.156713   

   Valor Presente  
0     -100.000000  
1        1.527075  
2        1.524899  
3        1.524899  
4        1.524899  
5        1.525282  
6        1.524899  
7       84.086142  
La Tasa Interna de Retorno (TIR) es: -4.71%


In [12]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy.optimize import newton

def add_months(date, months):
    month = date.month - 1 + months
    year = date.year + month // 12
    month = month % 12 + 1
    day = min(date.day, [31, 29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31][month - 1])
    return date.replace(year=year, month=month, day=day)

def xirr(transactions):
    days = [(date - transactions[0][0]).days for date, _ in transactions]
    totals = [total for _, total in transactions]

    def present_value(rate):
        total_pv = sum([total / (1 + rate)**(day / 365.0) for day, total in zip(days, totals)])
        return total_pv

    irr = newton(lambda r: present_value(r), 0.1)
    return irr

def fechas_pago_cupon(fecha_compra_str, fecha_emision_str, fecha_registro_str, periodicidad, tasa_base, spread, precio, precio_compra, tasa):
    periodos_por_año = {
        'TV': 4
    }

    n = periodos_por_año.get(periodicidad, None)
    if n is None:
        return "Periodicidad no reconocida"

    fecha_compra = datetime.strptime(fecha_compra_str, '%m/%d/%Y').date()
    fecha_emision = datetime.strptime(fecha_emision_str, '%m/%d/%Y').date()
    fecha_registro = datetime.strptime(str(fecha_registro_str), '%Y%m%d').date()

    fechas_pago = [fecha_compra]

    while fecha_emision < fecha_registro:
        if fecha_emision > fecha_compra:
            fechas_pago.append(fecha_emision)
        fecha_emision = add_months(fecha_emision, 12 // n)

    fechas_pago.append(fecha_registro)

    dias = [0]
    for i in range(1, len(fechas_pago)):
        diferencia = (fechas_pago[i] - fechas_pago[i-1]).days
        dias.append(diferencia)

    TP = (1 + spread)**(1/n) - 1
    flujos = [precio_compra * -100]
    flujos.extend([TP * 100] * (len(fechas_pago) - 2))
    flujos.append(TP * 100 + precio * 100)

    TP_tasa = (1 + tasa)**(1/n) - 1

    df = pd.DataFrame({
        'Conteo de Flujos': range(len(fechas_pago)),
        'Fecha de Pago': fechas_pago,
        'Días': dias,
        'Días/Tasa Base': [dia / tasa_base for dia in dias],
        'Flujos': flujos
    })

    df['Valor Presente'] = df['Flujos'] / (1 + TP_tasa)**df['Días/Tasa Base']

    transactions = list(zip(df['Fecha de Pago'], df['Flujos']))
    tir_resultante = xirr(transactions)

    return df, tir_resultante

# Ejemplo de uso
ejemplo_df, tir_resultante = fechas_pago_cupon('2/18/2022', '2/4/2020', '20230814', 'TV', 365, 0.0629, 0.8262, 1, 0.1303)
print(ejemplo_df)
print(f"La Tasa Interna de Retorno (TIR) es: {tir_resultante * 100:.2f}%")


   Conteo de Flujos Fecha de Pago  Días  Días/Tasa Base      Flujos  \
0                 0    2022-02-18     0        0.000000 -100.000000   
1                 1    2022-05-04    75        0.205479    1.536713   
2                 2    2022-08-04    92        0.252055    1.536713   
3                 3    2022-11-04    92        0.252055    1.536713   
4                 4    2023-02-04    92        0.252055    1.536713   
5                 5    2023-05-04    89        0.243836    1.536713   
6                 6    2023-08-04    92        0.252055    1.536713   
7                 7    2023-08-14    10        0.027397   84.156713   

   Valor Presente  
0     -100.000000  
1        1.527075  
2        1.524899  
3        1.524899  
4        1.524899  
5        1.525282  
6        1.524899  
7       84.086142  
La Tasa Interna de Retorno (TIR) es: -4.71%
